# Instrucciones
Apretar 'Run' > 'Run All Cells', para ejecutar el código.

Luego al final de la página se encuentra el gráfico interactivo.

In [1]:
import ipympl
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np;
import blackscholes as bs
from IPython.display import display, clear_output
import plotly.graph_objs as go

In [2]:
tipo_grafico = widgets.Dropdown(
    description='Tipo:',
    layout={'width': 'max-content'},
    options=[
        "Put",
        "Call",
        "Put 3D",
        "Call 3D",
        # "Put 3D (Interactivo)",
        # "Call 3D (Interactivo)",
        # "Straddle",
        # "Strangle",
        ])

In [3]:
sigma_widget = widgets.FloatSlider(
    value=35,
    min=0,
    max=100.0,
    step=0.1,
    description=r'\(\sigma\)',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
S_min = widgets.BoundedFloatText(
    value=80.0,
    min=0.0,
    max=1000.0,
    step=0.1,
    description=r'\(S_{min}\)',
    disabled=False,
    continuous_update=True,
    readout_format='.1f',
)
S_max = widgets.BoundedFloatText(
    value=120.0,
    min=0.0,
    max=1000.0,
    step=0.1,
    description=r'\(S_{max}\)',
    disabled=False,
    continuous_update=True,
    readout_format='.1f',
)

E = widgets.BoundedFloatText(
    value=100.0,
    min=0.0,
    max=100.0,
    step=0.1,
    description=r'\(E_T\)',
    disabled=False,
    continuous_update=True,
    readout_format='.1f',
)

r_free = widgets.FloatSlider(
    value=3.5,
    min=0.0,
    max=100.0,
    step=0.1,
    description=r'\(r_f\)',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [4]:
def plotting(r_f, sigma, s_min, s_max, E, tipo):
    S_T = E     # Strike price
    sigma = sigma_widget.value/100  # 15% Volatility
    q = 0.0       # 0% Annual Dividend Yield
    r_f = r_f/100

    S_range = np.linspace(s_min+0.001, s_max, 100)  # Range of asset prices
    T_values = [0.05, 0.25, 0.5, 1.0]  # Different times to maturity
    if tipo == "Put":
        plt.figure(figsize=(6, 4))

        for T in T_values:
            put_prices = []
            for S in S_range:
                put = bs.BlackScholesPut(S, S_T, T, r_f, sigma, q)
                put_prices.append(put.price())
            plt.plot(S_range, put_prices, label=f'T = {T}')
        intrinsic_values = np.maximum(S_T - S_range, 0)
        plt.plot(S_range, intrinsic_values, 'k--', label='Valor intrínsico')
        plt.xlabel('Asset Price (S)')
        plt.ylabel('Put Option Price')
        plt.title('Put Option Price vs Asset Price for Different Times to Maturity')
        plt.legend()
        plt.grid(True)
        plt.show()
    elif tipo == "Call":
        plt.figure(figsize=(6, 4))

        for T in T_values:
            call_prices = []
            for S in S_range:
                call = bs.BlackScholesCall(S, S_T, T, r_f, sigma, q)
                call_prices.append(call.price())
            plt.plot(S_range, call_prices, label=f'T = {T}')
        intrinsic_values = np.maximum(S_range - S_T, 0)
        plt.plot(S_range, intrinsic_values, 'k--', label='Valor intrínsico')
        plt.xlabel('Asset Price (S)')
        plt.ylabel('Call Option Price')
        plt.title('Call Option Price vs Asset Price for Different Times to Maturity')
        plt.legend()
        plt.grid(True)
        plt.show()
    elif tipo == "Put 3D":
        T_values.insert(0, 0.0001)
        S_grid, T_grid = np.meshgrid(S_range, T_values)
        P_grid = np.zeros_like(S_grid)
        
        for i in range(S_grid.shape[0]):
            for j in range(S_grid.shape[1]):
                S = S_grid[i, j]
                T = T_grid[i, j]
                put = bs.BlackScholesPut(S, S_T, T, r_f, sigma, q)
                P_grid[i, j] = put.price()
        
        fig = go.Figure(data=[go.Surface(z=P_grid, x=S_grid, y=T_grid, colorscale='Viridis')])
        fig.update_layout(
            title='Superficie de Precio de Opción Put (Interactivo)',
            scene=dict(
                xaxis_title='Precio del Activo (S)',
                yaxis_title='Tiempo al Vencimiento (T)',
                zaxis_title='Precio Opción Put'
            ),
            width=600,
            height=600
        )
        fig.show()
    elif tipo == "Call 3D":     
        T_values.insert(0, 0.0001)
        S_grid, T_grid = np.meshgrid(S_range, T_values)
        C_grid = np.zeros_like(S_grid)
        
        
        for i in range(S_grid.shape[0]):
            for j in range(S_grid.shape[1]):
                S = S_grid[i, j]
                T = T_grid[i, j]
                call = bs.BlackScholesCall(S, S_T, T, r_f, sigma, q)
                C_grid[i, j] = call.price()
        
        fig = go.Figure(data=[go.Surface(z=C_grid, x=S_grid, y=T_grid, colorscale='Viridis')])
        fig.update_layout(
            title='Superficie de Precio de Opción Call (Interactivo)',
            scene=dict(
                xaxis_title='Precio del Activo (S)',
                yaxis_title='Tiempo al Vencimiento (T)',
                zaxis_title='Precio Opción Call'
            ),
            width=500,
            height=500
        )
        fig.show()


In [5]:
plot_output = widgets.Output()
plot_output.layout.height = "800px"  # Set consistent height
update_button = widgets.Button(description="Actualizar gráfico", button_style='primary')

def on_update_button_clicked(b):
    with plot_output:
        clear_output(wait=True)
        plotting(
            r_f=r_free.value,
            sigma=sigma_widget.value,
            s_min=S_min.value,
            s_max=S_max.value,
            E=E.value,
            tipo=tipo_grafico.value
        )

update_button.on_click(on_update_button_clicked)
button_box = widgets.HBox([update_button])
button_box.layout.justify_content = 'center'

In [6]:
controls = widgets.GridBox(
    [sigma_widget, r_free, S_min, S_max, E, tipo_grafico],
    layout=widgets.Layout(
        grid_template_columns="repeat(2, 250px)",
        grid_gap="10px 50px",
    )
)

In [7]:
%%time
display(controls, button_box, plot_output)

GridBox(children=(FloatSlider(value=35.0, description='\\(\\sigma\\)', readout_format='.1f'), FloatSlider(valu…

Output(layout=Layout(height='800px'))

CPU times: total: 0 ns
Wall time: 13 ms
